In [4]:
import os
os.chdir('../')  # Moving up one directory to the root
!python --version
os.environ['WRAPT_DISABLE_EXTENSIONS'] = 'true'
print("done")

Python 3.12.7
done


In [5]:
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from main import SentimentAnalyzer
from models.sentiment_model import EnhancedDistilBertForSentiment, ModelTrainer
from config.model_config import ModelConfig
import matplotlib.pyplot as plt
import seaborn as sns
print("done")

2024-12-02 14:43:31.737035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


done


In [6]:
# Initialize analyzer
analyzer = SentimentAnalyzer()

# Process and show preprocessing examples
train_texts, val_texts, train_labels, val_labels = analyzer.process_data()
print("done")


Preprocessing Examples:

Original: Great service and amazing food!
Processed: Great service and amazing food!

Original: Terrible experience, would not recommend.
Processed: Terrible experience, would not_NEG recommend.

Original: The food was okay, but the service could be better.
Processed: The food was okay, but the service could be better.

Original: Yeah right, like that's going to work...
Processed: Yeah right, like that is going to work ELLIPSIS  _SARC_yeah right

Original: Thanks a lot... now everything is broken 🙄
Processed: Thanks a lot ELLIPSIS  now everything is broken 🙄 _SARC_thanks a lot (contextual)

Original: Obviously this is the best restaurant ever...
Processed: Obviously this is the best restaurant ever ELLIPSIS 
done


In [7]:
# Create the analyzer instance first
analyzer = SentimentAnalyzer()

# Then initialize the model
analyzer.initialize_model()
analyzer.initialize_model()

# Show model configuration
print("Model Configuration:")
print(f"BERT Model: {ModelConfig.BERT_MODEL}")
print(f"Learning Rate: {ModelConfig.LEARNING_RATE}")
print(f"Batch Size: {ModelConfig.BATCH_SIZE}")
print(f"Max Length: {ModelConfig.MAX_LENGTH}")
print("done")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.

Model Configuration:
BERT Model: distilbert-base-uncased
Learning Rate: 1e-05
Batch Size: 32
Max Length: 200
done


In [8]:
# Create a dummy input to build the model
dummy_input = analyzer.tokenizer(
    "This is a dummy text", 
    return_tensors='tf',
    padding=True,
    truncation=True,
    max_length=ModelConfig.MAX_LENGTH
)

# Build the model by passing dummy input
_ = analyzer.model(dummy_input)
analyzer.model.summary()
print("done")

Model: "enhanced_distil_bert_for_sentiment_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_distil_bert_model_1 (TF  multiple                  66362880  
 DistilBertModel)                                                
                                                                 
 bidirectional_1 (Bidirecti  multiple                  1154400   
 onal)                                                           
                                                                 
 dense_5 (Dense)             multiple                  30048     
                                                                 
 dense_6 (Dense)             multiple                  30048     
                                                                 
 dense_7 (Dense)             multiple                  30048     
                                                                 
 dense_8 (Dense)             m

In [9]:
# Train model

history = analyzer.train()

# training visualization
plt.figure(figsize=(15, 5))
# Loss plots
plt.subplot(1, 3, 1)
for metric in ['sentiment_loss', 'sarcasm_loss', 'negation_loss', 'polarity_loss']:
    if metric in history.history:
        plt.plot(history.history[metric], label=metric)
plt.title('Model Losses')
plt.legend()

# Accuracy plots
plt.subplot(1, 3, 2)
for metric in ['sentiment_accuracy', 'sarcasm_accuracy', 'negation_accuracy']:
    if metric in history.history:
        plt.plot(history.history[metric], label=metric)
plt.title('Model Accuracies')
plt.legend()

# MAE for polarity
plt.subplot(1, 3, 3)
if 'polarity_mae' in history.history:
    plt.plot(history.history['polarity_mae'], label='polarity_mae')
plt.title('Polarity MAE')
plt.legend()
plt.show()
print("done")

Epoch 1/5
 91/169 [===============>..............] - ETA: 1:02:40 - loss: 1.4326 - negation_loss: 0.2959 - polarity_loss: 0.1679 - sarcasm_loss: 0.2713 - sentiment_loss: 1.0986 - negation_accuracy: 0.9856 - polarity_mae: 0.3291 - sarcasm_accuracy: 0.9959 - sentiment_accuracy: 0.3743

Exception ignored in: Traceback (most recent call last):
'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
  File "/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/xc/qkmv0r456dn4gsy6rfgnpns80000gn/T/ipykernel_47201/372926428.py", line 3, in <module>
    history = analyzer.train()
              ^^^^^^^^^^^^^^^^
  File "/Users/tayebekavousi/Desktop/sentimentAnalysisPortfolio/main.py", line 95, in train
    return self.trainer.train(train_dataset, val_dataset)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tayebekavousi/Desktop/sentimentAnalysisPortfolio/models/sentiment_model.py", line 114, in train
    history = self.model.fit(
              ^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/tf_ker

KeyboardInterrupt: 

In [ ]:
test_cases = {
   'positive': [
       "The service was exceptional! The staff went above and beyond, the food was perfectly cooked, and the ambiance was wonderful. Everything from appetizers to dessert was a delight. Will definitely return!",
       "What an amazing dining experience! Every dish was beautifully presented, flavors were incredible, and the chef's special menu was innovative. Service was impeccable and attentive.",
       "First time here and absolutely loved it! The seafood pasta was fresh and flavorful, wine selection was excellent, and the outdoor seating area is gorgeous. Worth every penny!",
       "Best restaurant in the city! Incredible selection of dishes, outstanding cocktails, and the desserts are to die for. The atmosphere is perfect for both casual dining and special occasions.",
       "Phenomenal experience from start to finish! The steak was cooked to perfection, sides were creative and delicious, and the service was top-notch. Can't wait to come back!"
   ],
   'negative': [
       "Terrible experience. Food was cold, service was slow, and prices were ridiculous for what we got. The waiter was rude and unattentive. Definitely won't be coming back to this place.",
       "Disappointing dinner. Everything was bland, overcooked, and overpriced. Waited 45 minutes for our main course and the restaurant wasn't even busy. Save your money and go elsewhere.",
       "Worst dining experience ever! The food was inedible, the place was dirty, and we found a hair in our salad. Management didn't even care when we complained. Avoid at all costs!",
       "Complete waste of money. The portions were tiny, food was mediocre at best, and the service was awful. The place was too noisy and the tables were uncomfortably close together.",
       "Extremely disappointing. Food arrived cold, drinks were watered down, and the place was unclean. Staff seemed untrained and disinterested. Definitely not worth the high prices."
   ],
   'neutral': [
       "The food was okay, nothing special. Service was standard, and prices were what you'd expect. The place was clean but basic. Might return if in the area, but wouldn't go out of my way.",
       "Average experience overall. Some dishes were good, others were mediocre. Service was acceptable but not memorable. The atmosphere was typical for this type of restaurant.",
       "Had dinner here last night. Food came on time, taste was decent. Prices are standard for the area. The place is what you'd expect from a casual dining restaurant.",
       "Regular restaurant experience. The menu has typical options, service was neither good nor bad. Ambiance is basic. It's fine for a quick meal but don't expect anything extraordinary.",
       "Standard dining experience. Food was edible but not remarkable. Wait staff were polite but busy. Prices are average for what you get. It's an okay option if you're in the neighborhood."
   ],
   'sarcasm': [
       "Oh yeah, waiting an hour for cold food is EXACTLY what I wanted for my birthday. The rude service really made the experience special. Just perfect! 🙄",
       "Wow, thanks SO MUCH for the amazing experience of watching other tables get served while we waited. The cold food was definitely worth the two-hour wait! 😒",
       "Sure, charging $50 for microwaved pasta is totally reasonable! And the waiter's attitude? Cherry on top! Obviously the best restaurant ever... 🙄",
       "Yeah right, this place is definitely deserving of its 'premium' status. Nothing says luxury like paper napkins and plastic cups. Simply magnificent... 😏",
       "Just what I needed - overpriced, undercooked food with a side of attitude! The sticky tables really added to the 'authentic' experience. Thanks a lot! 🙄"
   ],
   'negation': [
       "The food was not fresh at all. Service wasn't good either. The ambiance didn't match the prices, and the manager wouldn't address our concerns.",
       "Cannot recommend this place. The menu isn't extensive, prices aren't reasonable, and the staff didn't seem to care about customer service at all.",
       "The restaurant didn't meet our expectations. Food wasn't hot when served, flavors weren't balanced, and the waiter never checked on our table.",
       "This place is not worth visiting. The food wasn't properly cooked, service wasn't professional, and the atmosphere wasn't pleasant at all.",
       "The chef couldn't cook properly. The staff didn't know the menu, and the manager wouldn't accept any feedback. Never returning here."
   ],
   'multipolar': [
       "Great appetizers but terrible main courses. The service was excellent, yet the prices were outrageous. Beautiful decor but unbearably noisy.",
       "The food was delicious but the service was awful. Love the atmosphere, however the cleanliness needs work. Amazing cocktails but terrible wine selection.",
       "Fantastic starters but disappointing entrees. The staff was friendly but extremely slow. Beautiful venue but uncomfortable seating arrangements.",
       "Excellent food quality but tiny portions. The bartender was amazing, yet the servers were rude. Love the concept but poor execution.",
       "Perfect location but horrible parking. The food was tasty but overpriced. Friendly host but inattentive waiters. Mixed feelings about this place."
   ],
   'long_reviews': [
       "Had a truly memorable experience at this establishment last weekend. The atmosphere immediately draws you in with its perfect blend of modern and classic decor. Started with their signature appetizers - the calamari was perfectly crispy and the truffle fries were addictive. Main courses were even better; the sea bass was cooked to perfection and practically melted in the mouth. Wine pairing suggestions were spot-on. Service was attentive without being intrusive. The dessert menu was innovative - tried their deconstructed cheesecake which was a delightful surprise. Pricing was on the higher side but justified by the quality. Will definitely return!",
       "After months of trying to get a reservation, our experience was unfortunately disappointing. While the ambiance was elegant with stunning city views, everything else fell short. Service was painfully slow and disorganized - waited 45 minutes between appetizers and main course. The food arrived lukewarm and wasn't worth the premium pricing. The wagyu steak was overcooked despite requesting medium-rare, and the seafood platter didn't taste fresh. When we raised these issues, the manager seemed indifferent. The cocktails were the only saving grace, but not enough to warrant a second visit. Save your money and try somewhere else.",
       "The restaurant offers a standard dining experience with some hits and misses. Decor is contemporary but could use updating. The menu features typical Italian dishes - some executed well, others needing improvement. Service was adequate but not exceptional, and prices align with similar establishments in the area. The pasta dishes were cooked properly, though sauces lacked depth. Wine list is decent but not extensive. Seating is comfortable, noise level manageable. Parking can be challenging during peak hours. It's a safe choice for casual dining but doesn't stand out in any particular way.",
       "Oh, what a 'delightful' evening! 🙄 Absolutely LOVED waiting 45 minutes past our reservation time, especially appreciated being squeezed into a tiny table right by the kitchen door. The waiter's complete lack of knowledge about the menu was simply CHARMING! And how thoughtful of them to serve us cold food at premium prices - such a unique dining concept! The manager's defensive attitude when we complained was the cherry on top of this 'exceptional' experience. But hey, at least the water glasses were full... when we could find someone to fill them. Just perfect for our anniversary celebration!",
       "This restaurant perfectly embodies both excellence and disappointment. The appetizers were absolutely stunning - perfectly seasoned and beautifully presented. However, the main courses were a complete letdown with overcooked proteins and bland sauces. The sommelier was incredibly knowledgeable and helpful, but our server was inattentive and seemed disinterested. The venue itself is gorgeous with amazing waterfront views, yet the tables are uncomfortably close together. The dessert menu looked promising but the execution was inconsistent. Mixed feelings about recommending this place despite its potential."
   ]
}

print("\nComprehensive Sentiment Analysis:")
for category, cases in test_cases.items():
    print(f"\n{category.upper()} CASES:")
    print("-" * 80)
    
    for text in cases:
        results = analyzer.predict(text)
        print(f"\nText: {text}")
        print(f"Sentiment: {max(results['sentiment'].items(), key=lambda x: x[1])[0]}")
        print(f"Sentiment Distribution: neg={results['sentiment']['negative']:.2f}, "
              f"neu={results['sentiment']['neutral']:.2f}, "
              f"pos={results['sentiment']['positive']:.2f}")
        print(f"Sarcasm: {results['sarcasm']['detected']}")
        print(f"Negation: {results['negation']['detected']}")
        print(f"Multipolarity Score: {results['multipolarity']['score']:.2f}")
        print(f"Is Multipolar: {results['multipolarity']['is_multipolar']}")
        print("-" * 50)

print("done")